In [1]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from scipy.integrate import dblquad
from scipy.constants import golden_ratio
from scipy.interpolate import RegularGridInterpolator

import astropy.constants as const
import time
import astropy.units as u

from astropy.cosmology import LambdaCDM
cosmo = LambdaCDM(H0=70, Om0=0.3, Ode0=0.7)

import gwent
import gwent.binary as binary
import gwent.detector as detector
import gwent.snr as snr

#Turn off warnings for tutorial
import warnings
warnings.filterwarnings('ignore')

In [2]:
#set matplotlub parameters
def get_fig_size(width=7,scale=2.0):
    #width = 3.36 # 242 pt
    base_size = np.array([1, 1/scale/golden_ratio])
    fig_size = width * base_size
    return(fig_size)
mpl.rcParams['figure.dpi'] = 300
#mpl.rcParams['figure.figsize'] = get_fig_size()
mpl.rcParams['text.usetex'] = True
mpl.rc('font',**{'family':'serif','serif':['Times New Roman']})
mpl.rcParams['lines.linewidth'] = 1.3
mpl.rcParams['axes.labelsize'] = 12
mpl.rcParams['xtick.labelsize'] = 10
mpl.rcParams['ytick.labelsize'] = 10
mpl.rcParams['legend.fontsize'] = 8

The following constants will be useful for our calculations.

In [3]:
#Redshifts
z0 = 1e-2
z_min = 30.0
z_max = 5e3
sigmaM_squared = 3.6*1e-5

t0 = cosmo.age(z0).value #present age of universe in years in Gyr

detector = 'ET'

M_samples = np.load('SNRcalc/'+detector+'_SNR_IMRPhenomXAS_Mtot_array.npy')
q_samples = np.load('SNRcalc/'+detector+'_SNR_IMRPhenomXAS_q_array.npy')

We first code for $p_\mathrm{det}$ by considering the function $P(\omega)$

\begin{equation}
p_\mathrm{det} = P(\omega) = a^{(n)}_2[(1-\omega/\alpha^{(n)})^2] + a^{(n)}_4[(1-\omega/\alpha^{(n)})^4] + a^{(n)}_8[(1-\omega/\alpha^{(n)})^8] + (1 - a^{(n)}_2 - a^{(n)}_4 - a^{(n)}_8)[(1-\omega/\alpha^{(n)})^{10}],
\end{equation}

where the coefficients $a^{(n)}_i$ are fit parameters that vary depending on the number of detectors $n$ in the system, and $\alpha^{(n)}$ is the maximum value of $\omega$ for a given number of detectors. Since the LISA observatory is a three-detector system, our coefficients take the values: $a^{(3)}_2 = 1.19549$, $a^{(3)}_4 = 1.61758$, $a^{(3)}_8 = -4.87024$, $\alpha^{(3)} = 1.4$.

In [4]:
def cumulative_P(w):
    a2 = .374222
    a4 = 2.04216
    a8 = -2.63948
    alpha = 1.0
    
    w_final = np.where(w >= alpha, alpha, w)
    signal = w_final/alpha
    
    P = a2*(np.power(1 - signal, 2.0)) + a4*(np.power(1 - signal, 4.)) + a8*(np.power(1 - signal,8.0)) + (1 - a2 - a4 - a8)*(np.power(1 - signal, 10.0))
    
    return P

Note here that $w = \rho_\mathrm{thr}/\rho_\mathrm{opt}(M_1, M_2, z)$, where $\rho_\mathrm{thr} = 8$ is the minimum SNR usually accepted for a significant signal. In order to compute for $\rho_\mathrm{opt}(M_1, M_2, z)$, we utilize the GW observatory simulation package `gwent`. 

(Source initialization will now be done externally.)

In [5]:
def trapz_double(func, M_min, M_max, n=100):
    
    x_list = np.logspace(np.log10(M_min), np.log10(M_max), n)
    y_list = np.logspace(np.log10(M_min), np.log10(M_max), n)
    
    X, Y= np.meshgrid(x_list, y_list, indexing='ij', sparse=True)
    
    f_comp = func(X, Y)

    f_mean = f_comp.sum()/(len(f_comp)**2.0)
    value = ((M_max - M_min)**2.0)*f_mean

    return value

In [6]:
"""
Power law mass distribution model. Takes in PBH mass M as a variable. Reference mass M_c and width sigma are model parameters.
"""

def pl(M, M_min, M_max, gamma):
    norm = gamma / (M_max**gamma - M_min**gamma)
    return norm * M**(gamma - 1.0) 

In [7]:
"""
Average local number of neighbors N(y) in radius y for any pair of PBHs with masses M1 and M2 
given the average mass M_ave of the distribution and initial PBH abundance f.
"""
def nbar(M_total, f, M_ave):
    return (M_total/M_ave)*(f/(f + np.sqrt(sigmaM_squared)))

In [8]:
"""
C(f): fitting function used to approximate the supression behaviour of supp1 at middling values of f.
Takes in mass distribution average and variance as parameters.
"""

import scipy.special as sc

def cfit(f, M_ave, M_var):
    fs = np.power(f, 2.0)/sigmaM_squared
    hyp = (0.255*sc.hyperu(21.0/74.0, 0.5, (5.0/6.0)*fs))**(-74.0/21.0)
    
    return (M_var/np.power(M_ave, 2.0))*fs/(hyp - 1.0)

In [9]:
"""
S1(M1, M2, f): merger rate suppression due to DM and matter inhomogeneities in the local region of the binary.
"""
def supp1(M_total, f, M_ave, M_var):
    factor = np.power((M_var/np.power(M_ave, 2.0))/(nbar(M_total, f, M_ave) + cfit(f, M_ave, M_var)) 
                    + sigmaM_squared/np.power(f,2.0), -21.0/74.0)
    
    return 1.42*factor*np.exp(-nbar(M_total, f, M_ave))

In [10]:
def supp2(z, f):
    t = cosmo.age(z).value
    x = np.power(t.flatten()/t0, 0.44)*0.85*f
    
    suppfactor = (9.6*1e-3)*np.power(x, -0.65)*np.exp(0.03*(np.log(x))**2.0)
    
    return [min(1.0, supp) for supp in suppfactor]

In [11]:
"""
dR(M1, M2, z, f): differential merger rate of PBHs.
"""
def merger_rate_log(M1, M2, z, f, M_min, M_max, gamma):
    t = cosmo.age(z).value
    
    M_tot = M1 + M2
    M_square = M1*M2
    
    norm = gamma / (M_max**gamma - M_min**gamma)
    M_ave = ((gamma + 1)**-1.0) * norm * (M_max**(gamma + 1.) - M_min**(gamma + 1.))
    M_var = ((gamma + 2)**-1.0) * norm * (M_max**(gamma + 2.) - M_min**(gamma + 2.))
    
    f_dep = np.power(f, 53.0/37.0)
    
    M_factor = np.power(M_square/np.power(M_tot, 2.0), -34.0/37.0)
    M_dep = np.power(M_tot, -32.0/37.0)
    
    t_dep = np.power(t/t0, -34.0/37.0)
    
    supp = supp1(M_tot, f, M_ave, M_var)*supp2(z, f)
    
    dists = pl(M1, M_min, M_max, gamma)*pl(M2, M_min, M_max, gamma)
    
    return (1.6*1e6)*f_dep*t_dep*M_factor*M_dep*supp*dists*(u.Gpc**-3.0)

In [12]:
def dN(M1, M2, z, f, M_min, M_max, gamma):
    dV = cosmo.differential_comoving_volume(z).to("Gpc^3 / sr")*4.0*np.pi*u.sr
    a = cosmo.scale_factor(z)
    
    M_tot = M1 + M2
    q_temp = M2/M1
    q = np.where(q_temp >= 1, q_temp, 1.0/q_temp) 
    
    pts = np.array([(ratio, mass) for mass, ratio in zip(M_tot.flatten(), q.flatten())])
    
    rho_thr = 8.0
    rho_opt = SNR_interp(pts)

    p_det = cumulative_P(rho_thr/rho_opt)
    merge = merger_rate_log(M1, M2, z, f, M_min, M_max, gamma)
    
    res = merge.flatten()*p_det*dV*a

    return res

In [13]:
def Ndet(M_min, M_max, z, f, gamma):
    integ = lambda M1, M2: dN(M1, M2, z, f, M_min, M_max, gamma)

    N_eval = trapz_double(integ, M_min, M_max, 200)
    
    return N_eval

We consider the mass range from $M_\mathrm{min} \sim 10\,M_\odot$ to $M_\mathrm{max} \approx 10^{10}\,M_\odot$ and the PBH abundance range $10^{-10} \leq f_\mathrm{PBH} \leq 10^0$.

In [14]:
from IPython.display import display, clear_output
import pandas as pd

M_ref = np.logspace(1, 9, 100) #M_sun
sigma = np.linspace(0.1, 5, 10)

fPBH_samples = np.logspace(-10.0, 0.0, M_ref.size)
z_space = np.logspace(np.log10(z0), np.log10(z_max), 100)

start_time = time.time()
loop_no = 0

for sig in sigma:
    gamma = -1.0 / sig
    N_array = np.zeros((M_ref.size, fPBH_samples.size, z_space.size))
    for z in z_space:
        SNR_extract = np.load('SNRcalc/'+detector+'_SNR_IMRPhenomXAS_z'+str(z)+'.npy')
        SNR_interp = RegularGridInterpolator((q_samples, M_samples), SNR_extract, method="nearest", bounds_error=False, fill_value=0)
        
        z_index = np.where(z_space == z)[0][0]
    
        mass_min_array = M_ref * np.exp(1.0 / gamma)
        mass_max_array = mass_min_array * 1e3
        iteration = 0
        for mass, mass_min, mass_max, in zip(M_ref, mass_min_array, mass_max_array):
            
            mass_index = np.where(M_ref == mass)[0][0]
    
            iteration += 1
            iter_percent = (iteration/M_ref.size)*100.0
    
            inner_iter = 0
            
            for fPBH in fPBH_samples:
                fPBH_index = np.where(fPBH_samples == fPBH)[0][0]
                
                N = Ndet(mass_min, mass_max, z, fPBH, gamma)
                N_array[mass_index][fPBH_index][z_index] = N
                inner_iter +=1
                clear_output(wait=True)
                display("Redshift z = "+str(z)+", gamma = "+str(gamma),
                        "Current loop progress: "+str(iter_percent)+"%, current mass: "+str(mass)+" solar masses")
                time.sleep(0)
                
    np.save('powerlaw_gamma_minus_Ndet/'+detector+'_powerlaw_fPBH-mass_sigma'+str(sig)+'_1yr.npy', N_array)
    
end_time = time.time()

print("Total runtime: ", end_time - start_time, " seconds")

'Redshift z = 5.076584485817265, gamma = -1.5517241379310345'

'Current loop progress: 100.0%, current mass: 1000000000.0 solar masses'

KeyboardInterrupt: 